#ME536 - Semester Project
İsmail Melih CANBOLAT - 2033199

#Imports
Required packages are imported.

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model

from sklearn.decomposition import PCA


In [ ]:
# # Download sample images from GitHub - For test purposes
# import os
# !rm *.jpg  # Remove previous images
# img_count = 9 # Number of images available at the link
# for i in range(img_count):
#     os.system("wget https://raw.githubusercontent.com/melih-canbolat/"+
#               "ME536-Project/main/Webots-Simulation/controllers/slave/"+
#               "captures/img"+str(i)+".jpg")
#     print("img"+str(i)+".jpg")


In [ ]:
def show(im):
    """
    Simple function to display an image
    """
    plt.imshow(im, cmap = 'gray')
    plt.show()


In [ ]:
class Item:
    """
    This class is used to create an instance for each detected item/object on the image.
    """
    def __init__(self, stats, image):
        self.x = stats[0]       # leftmost (x) coordinate
        self.y = stats[1]       # topmost (y) coordinate
        self.width = stats[2]   # horizontal size of the bounding box
        self.height = stats[3]  # vertical size of the bounding box
        self.frame(image)       # Extract the Region of Interest from the image

    def frame(self, src):
        i = src[self.y:self.y+self.height+1, self.x:self.x+self.width+1]  # Extract the ROI from orginal capture
        self.img = cv.cvtColor(i, cv.COLOR_BGR2RGB)  # RGB version of the original image of the item
        
        # === UNUSED PART - REMOVE === #
        # Upscale the image with certain interpolation algorithm to obtain smoother & larger image
        # self.resized = cv.resize(img,None,fx=3, fy=3, interpolation = cv.INTER_CUBIC)  # Slower
        # img_resized = cv.resize(img,None,fx=3, fy=3, interpolation = cv.INTER_LINEAR)  # Faster
        # img_resized = cv.resize(img,(224,224), interpolation = cv.INTER_LINEAR)  # Faster
        # img_padded = cv.copyMakeBorder(img_resized,10,10,10,10,cv.BORDER_CONSTANT,value=0)  # Zero padding
        # self.img_resized = img_resized
        # self.img_padded = img_padded  # Padded image will be used for feature extraction.



In [ ]:
detected = []  # List to store instances created for each detected item
dataset = np.zeros((25088,1), dtype="float32")  # Numpy array of feature vectors

#Image Processing
Does following:

*   Convert the image to grayscale
*   Apply threshold to convert to binary image
*   Invert binary image so that objects of interest are white
*   Apply erosion followed by dilation to remove noise
*   Use Connected Component Labeling algorithm to detect objects on the image
*   Create an instance for each detected item & store them in a list




In [ ]:
def process(image):
    """
    Accepts the captured image and processes that image to detect objects 
    in it. Then, creates an instance Item class for each detected item and
    stores them in the list: "detected[]"
    """
    gimg = cv.cvtColor(image, cv.COLOR_BGR2GRAY)                                # Convert to grayscale
    ret, thresh = cv.threshold(gimg, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)   # Apply OTSU threshold
    th1 = np.invert(thresh)                                                     # Invert binary image so that objects of interest are white
    kernel = np.ones((2,2),np.uint8)                                            # Define a kernel to be used for morphological transformations
    img_final = cv.morphologyEx(th1, cv.MORPH_OPEN, kernel)                     # Erosion followed by Dilation - removes noise
    # erosion = cv.erode(th,kernel,iterations = 1)    # Erosion only
    # dilation = cv.dilate(th,kernel,iterations = 1)  # Dilation only
    n, labels, stats, centroids = cv.connectedComponentsWithStats(img_final)    # Connected Component Labeling (with stats)

    # Create an instance for each detected item (except background) & add it to the list
    for i in range(1, n):
        detected.append(Item(stats[i], image))


    # === Draw bounding boxes - for debugging purposes === #
    if 0:
        img_boxed = cv.cvtColor(img_final, cv.COLOR_GRAY2BGR)
        for i in detected:
            cv.rectangle(img_boxed, (i.x, i.y), (i.x+i.width, i.y+i.height), (255,0,0), 1)
        show(img_boxed)


    # ==== Following can be considered for improvements ==== #
    # img_blur = cv.blur(img,(10,10)) # Blur the image using a simple kernel of size 10x10
    # contours, hierarchy = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)  # Find the contours on a binary image
    # cv.drawContours(img, contours, -1, (0, 255, 0), 3)  # Draw contours on img

    return None


In [ ]:
# # === TEST SECTION === #
# captured = cv.imread("img0.jpg")  # Read an image (BGR)
# x = process(captured)  # Proces that image and update the list: "detected[]"
# print(x)

#Feature Extraction Using **C**onvolutional **N**eural **N**etworks
A model is created based on VGG19 so that a feature vector is extracted from an intermediate layer of the VGG19 network.  
Original RGB image of the detected object is fed into the network. And for each object, feature vectors are obtained.  

More info on VGG19:  
https://keras.io/api/applications/vgg/#vgg19-function


First, create the model:

In [ ]:
base_model = VGG19(weights='imagenet')  # Base model is VGG19 with pre-trained weights

# Do not use dense layers of VGG19. Get the output from "flatten" layer of VGG19.
# i.e. Extract feature vector from an intermediate layer of VGG19
model = Model(inputs=base_model.input, outputs=base_model.get_layer('flatten').output) 

# ===== From Example Code =====
# img_path = 'img0.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)
# feature_vector = model.predict(x)


In [ ]:
# model.summary()

In [ ]:
# # ===== Prepare an image to extract its Feature Vector with the ANN Model =====
# x = cv.cvtColor(detected[0].img_resized, cv.COLOR_GRAY2BGR)
# x = x.astype('float32')
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# feature_vector = model.predict(x)

In [ ]:
def analyze(x):
    """
    This function accepts an image(RGB) and returns feature vector for that image.
    """
    x = cv.resize(x, (224, 224), interpolation = cv.INTER_LINEAR);  # Resize the image so that its dimesions are suitable for VGG19
    x = x.astype('float32')  # Convert data type to float32
    x = np.expand_dims(x, axis=0)  # Expand dimesnions
    x = preprocess_input(x)  # Preproces the image for vgg19

    feature_vector = model.predict(x)  # Get the feature vector from the CNN

    return feature_vector


In [ ]:
captured = cv.imread("img8.jpg")  # Read an image (BGR)
process(captured)  # Proces that image and update the list: "detected[]"


# Data Analysis - Incomplete
Now that we have feature vectors, we can use them as data points in our dataset and determine if a newly added data is new or not.

Steps:

*   Dimension reduction of feature vectors using PCA
*   Clustering data
*   Detecting new cluster formation


In [ ]:
fv = analyze(detected[i].img)  # Analyze images in the detected list by feeding them into CNN and get the feature vector for them 
# dataset = np.append(dataset, np.transpose(fv), axis=1)  # Add the feature vector to dataset as a column vector


In [ ]:
pca = PCA(n_components=2)
pca.fit(X)

In [ ]:
u, s, vt = np.linalg.svd(dataset, full_matrices=True)
plt.plot(s, "k.")